## Operators and Algorighms

### A First Individual

In [5]:
import random
from deap import base
from deap import creator
from deap import tools

IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=IND_SIZE)

ind1 = toolbox.individual()
print(ind1)
print(ind1.fitness.valid)

[0.19058551952010994, 0.5458037745726918, 0.45180770454047825, 0.9300209237988409, 0.4240369785765895]
False


### Evaluation

In [7]:
def evaluate(individual):
    a = sum(individual)
    b = len(individual)
    return a, 1.0/b

ind1.fitness.values = evaluate(ind1)
print(ind1.fitness.valid)
print(ind1.fitness)

True
(2.5422549010087105, 0.2)


### Mutation

In [10]:
mutant = toolbox.clone(ind1)
ind2, = tools.mutGaussian(mutant, mu=0.0, sigma=0.2, indpb=0.2)
del mutant.fitness.values

print(ind2 is mutant)
print(mutant is ind1)

True
False


### Crossover

Note Just as a remark on the language, the form toolbox.clone([ind1, ind2]) cannot be used because if ind1 and ind2 are referring to the same location in memory (the same individual) there will be a single independent copy of the individual and the second one will be a reference to this same independent copy. This is caused by the mechanism that prevents recursive loops. The first time the individual is seen, it is put in the “memo” dictionary, the next time it is seen the deep copy stops for that object and puts a reference to that previously created deep copy. Care should be taken when deep copying containers.

In [13]:
child1, child2 = [toolbox.clone(ind) for ind in (ind1,ind2)]
tools.cxBlend(child1, child2, 0.5)
del child1.fitness.values
del child2.fitness.values

### Selection

Warning It is very important here to note that the selection operators does not duplicate any individual during the selection process. If an individual is selected twice and one of either object is modified, the other will also be modified. Only a reference to the individual is copied. Just like every other operator it selects and only selects.

In [17]:
# parameter: iterable container of individuals and the number of individuals to select
selected = tools.selBest([child1,child2], 2) 
print(child1 in selected)

# usually duplication of the entire population will be made after selection or before variation
offspring = [toolbox.clone(ind) for ind in selected]

True


### Using the Toolbox

In [18]:
toolbox = base.Toolbox()

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

#### Using the tools

In [ ]:
for g in range(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = map(toolbox.clone, offspring)

    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring

### Tool Decoration

In [ ]:
def checkBounds(min, max):
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in xrange(len(child)):
                    if child[i] > max:
                        child[i] = max
                    elif child[i] < min:
                        child[i] = min
            return offspring
        return wrapper
    return decorator

toolbox.register("mate", tools.cxBlend, alpha=0.2)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=2)

toolbox.decorate("mate", checkBounds(MIN, MAX))
toolbox.decorate("mutate", checkBounds(MIN, MAX))

### Variations

Variations allow to build simple algorithms using predefined small building blocks. In order to use a variation, the toolbox must be set to contain the required operators. For example in the lastly presented complete algorithm, the crossover and mutation are regrouped in the varAnd() function, this function requires the toolbox to contain the mate() and mutate() functions. This variation can be used to simplify the writing of an algorithm as follows.

In [ ]:
from deap import algorithms

for g in range(NGEN):
    # Select and clone the next generation individuals
    offspring = map(toolbox.clone, toolbox.select(pop, len(pop)))

    # Apply crossover and mutation on the offspring
    offspring = algorithms.varAnd(offspring, toolbox, CXPB, MUTPB)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring

### Algorithms

There are several algorithms implemented in the algorithms module. They are very simple and reflect the basic types of evolutionary algorithms present in the literature. The algorithms use a Toolbox as defined in the last sections. In order to setup a toolbox for an algorithm, you must register the desired operators under the specified names, refer to the documentation of the selected algorithm for more details. Once the toolbox is ready, it is time to launch the algorithm. The simple evolutionary algorithm takes 5 arguments, a _population_, a _toolbox_, a probability of mating each individual at each generation (_cxpb_), a probability of mutating each individual at each generation (_mutpb_) and a number of generations to accomplish (_ngen_).

In [ ]:
from deap import algorithms

algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50)